<a href="https://colab.research.google.com/github/arielkeren/relationai/blob/main/RelationAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shared

In [ ]:
!pip install tensorflowjs
!pip install TensorFlow==2.15.0
!pip install tensorflow-decision-forests==1.8.1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from typing import Callable, Tuple
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflowjs.converters import save_keras_model
from tensorflow.keras.backend import clear_session

In [ ]:
# The size of the set that the relation operates on.
SET_SIZE = 5

# The size of each of the generated datasets.
TRAIN_SIZE = 600000
VALIDATION_SIZE = 200000
TEST_SIZE = 200000

# The number of epochs of every model.
EPOCHS = 10
# The batch size used during training.
BATCH_SIZE = 32

# Type representing the returned datasets from functions.
Dataset = Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]
# Type representing the returned tuple of x and y arrays from functions.
LabeledData = Tuple[np.ndarray, np.ndarray]
# Type representing a function that generates a sample for a dataset.
GenerationFunction = Callable[[], np.ndarray]

In [ ]:
clear_session()

In [ ]:
def get_property_model() -> Sequential:
  """
  Returns a model for property classification.

  Returns:
    Sequential: The compiled model.
  """

  model = Sequential()

  model.add(Input(shape=(SET_SIZE * SET_SIZE,)))
  model.add(Dense(32, activation="relu"))
  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

  return model

def generate_binary_dataset(number_of_samples: int, generate_positive_case: GenerationFunction, generate_negative_case: GenerationFunction) -> LabeledData:
  """
  Generates a dataset for binary classification problems.

  Args:
    number_of_samples (int): The number of samples in the dataset.
    generate_positive_case (GenerationFunction): The function to generate the positive case.
    generate_negative_case (GenerationFunction): The function to generate the negative case.

  Returns:
    LabeledData: The generated dataset, split to x and y.
  """

  # Initialize the x and y arrays.
  x, y = [], []

  # Run for half the samples, each time adding one from each case.
  for _ in range(number_of_samples // 2):
    # Add a positive case.
    x.append(generate_positive_case())
    y.append(1)
    # Add a negative case.
    x.append(generate_negative_case())
    y.append(0)

  # Convert the arrays to Numpy arrays to manipulate them.
  x, y = np.array(x), np.array(y)

  # Randomize the arrays in the same way.
  indices = np.random.permutation(len(y))
  x, y = x[indices], y[indices]

  # Return the dataset.
  return x, y

def generate_full_binary_dataset(generate_positive_case: GenerationFunction, generate_negative_case: GenerationFunction) -> Dataset:
  """
  Generates a full dataset for binary classification problems, consisting of training, validation and test.

  Args:
    generate_positive_case (GenerationFunction): The function to generate the positive case.
    generate_negative_case (GenerationFunction): The function to generate the negative case.

  Returns:
    Dataset: The generated dataset, split to x and y.
  """

  # Generate the training dataset.
  x_train, y_train = generate_binary_dataset(TRAIN_SIZE, generate_positive_case, generate_negative_case)
  # Generate the validation dataset.
  x_val, y_val = generate_binary_dataset(VALIDATION_SIZE, generate_positive_case, generate_negative_case)
  # Generate the test dataset.
  x_test, y_test = generate_binary_dataset(TEST_SIZE, generate_positive_case, generate_negative_case)

  # Return the full dataset.
  return (x_train, y_train), (x_val, y_val), (x_test, y_test)

def generate_full_dataset(generate_case: Callable[[int], Tuple[np.ndarray, np.ndarray]]) -> Dataset:
  """
  Generates a full dataset, including training, validation and test.

  Args:
    generate_case (Callable[[], np.ndarray]): The function to generate a pair of x and y arrays.

  Returns:
    Dataset: The generated dataset.
  """

  x_train, y_train = generate_case(TRAIN_SIZE)
  x_val, y_val = generate_case(VALIDATION_SIZE)
  x_test, y_test = generate_case(TEST_SIZE)

  return (x_train, y_train), (x_val, y_val), (x_test, y_test)

def get_random_relation() -> np.ndarray:
  """
  Generates a random relation.

  Returns:
    np.ndarray: The generated relation.
  """

  # 0 means that the pair satisfies the relation, and 1 means that it does not.
  return np.random.randint(0, 2, (SET_SIZE, SET_SIZE))

def plot_loss(loss: np.ndarray, val_loss: np.ndarray) -> None:
  """
  Plots the training loss and the validation loss of a model.

  Args:
    loss (np.ndarray): The training loss of the model to plot.
    val_loss (np.ndarray): The validation loss of the model to plot.
  """

  plt.plot(loss, label="Training Loss")
  plt.plot(val_loss, label="Validation Loss")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend()
  plt.show()

def plot_accuracy(accuracy: np.ndarray, val_accuracy: np.ndarray) -> None:
  """
  Plots the training accuracy and the validation accuracy of a model.

  Args:
    loss (np.ndarray): The training accuracy of the model to plot.
    val_loss (np.ndarray): The validation accuracy of the model to plot.
  """

  plt.plot(accuracy, label="Training Accuracy")
  plt.plot(val_accuracy, label="Validation Accuracy")
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy")
  plt.legend()
  plt.show()

def operation_accuracy(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
  """
  Calculates the accuracy of an operation model.

  Args:
    y_true (tf.Tensor): The true values.
    y_pred (tf.Tensor): The predicted values.

  Returns:
    tf.Tensor: The accuracy of the operation model.
  """

  # Convert predictions to binary.
  y_pred_binary = tf.cast(y_pred >= 0.5, tf.bool)

  # Convert true values to boolean.
  y_true_binary = tf.cast(y_true, tf.bool)

  # Check if all the elements in each row are equal.
  correct_predictions = tf.reduce_all(tf.equal(y_pred_binary, y_true_binary), axis=1)

  # Compute the accuracy as the average of the correct predictions.
  return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

# Reflexivity

In [ ]:
def generate_reflexive_matrix() -> np.ndarray:
  """
  Generates a reflexive matrix.

  Returns:
    np.ndarray: The generated reflexive matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()
  # Fill the main diagonal with ones, so that for every element x, (x, x) satisfies the relation.
  np.fill_diagonal(matrix, 1)

  return matrix.flatten()

def generate_non_reflexive_matrix() -> np.ndarray:
  """
  Generates a non-reflexive matrix.

  Returns:
    np.ndarray: The generated non-reflexive matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()
  # Ensure that it is not reflexive, by randomly making pairs on the main diagonal not satisfy the relation.
  for _ in range(np.random.randint(1, 5)):
    # Each time, pick a random pair from the main diagonal and set it to 0.
    index = np.random.randint(0, SET_SIZE)
    matrix[index][index] = 0

  return matrix.flatten()

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_binary_dataset(generate_reflexive_matrix, generate_non_reflexive_matrix)

In [ ]:
model = get_property_model()

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["accuracy"], history.history["val_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/reflexivity')

# Irreflexivity

In [ ]:
def generate_irreflexive_matrix() -> np.ndarray:
  """
  Generates an irreflexive matrix.

  Returns:
    np.ndarray: The generated irreflexive matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()
  # Fill the main diagonal with zeros, so that for every element x, (x, x) does not satisfy the relation.
  np.fill_diagonal(matrix, 0)

  return matrix.flatten()

def generate_non_irreflexive_matrix() -> np.ndarray:
  """
  Generates a non-irreflexive matrix.

  Returns:
    np.ndarray: The generated non-irreflexive matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()
  # Ensure that it is not irreflexive, by randomly making pairs on the main diagonal satisfy the relation.
  for _ in range(np.random.randint(1, 5)):
    # Each time, pick a random pair from the main diagonal and set it to 1.
    i = np.random.randint(0, SET_SIZE)
    matrix[i][i] = 1

  return matrix.flatten()

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_binary_dataset(generate_irreflexive_matrix, generate_non_irreflexive_matrix)

In [ ]:
model = get_property_model()

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["accuracy"], history.history["val_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/irreflexivity')

# Symmetry

In [ ]:
def generate_symmetric_matrix() -> np.ndarray:
  """
  Generates a symmetric matrix.

  Returns:
    np.ndarray: The generated symmetric matrix.
  """

  # Create a random matrix.
  random_matrix = get_random_relation()
  # Copy only the values on the main diagonal or above to a new matrix.
  matrix = np.triu(random_matrix, 0)
  # Mirror the upper values to ensure that the matrix is symmetric.
  matrix += np.triu(random_matrix, 1).T

  return matrix.flatten()

def generate_non_symmetric_matrix() -> np.ndarray:
  """
  Generates a non-symmetric matrix.

  Returns:
    np.ndarray: The generated non-symmetric matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Ensure a non-symmetric matrix, by making some asymmetric pairs satisfy the relation.
  for _ in range(np.random.randint(1, 5)):
    # Choose 2 random pairs
    i, j = np.random.randint(0, SET_SIZE, 2)
    # Make them asymmetric
    if i != j:
      matrix[i][j] = 1 - matrix[j][i]

  return matrix.flatten()

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_binary_dataset(generate_symmetric_matrix, generate_non_symmetric_matrix)

In [ ]:
model = get_property_model()

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["accuracy"], history.history["val_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/symmetry')

# Asymmetry

In [ ]:
def generate_asymmetric_matrix() -> np.ndarray:
  """
  Generates an asymmetric matrix.

  Returns:
    np.ndarray: The generated asymmetric matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Make all pairs asymmetric
  for i in range(SET_SIZE):
    for j in range(SET_SIZE):
      if matrix[i][j]:
        matrix[j][i] = 0

  return matrix.flatten()

def generate_non_asymmetric_matrix() -> np.ndarray:
  """
  Generates a non-asymmetric matrix.

  Returns:
    np.ndarray: The generated non-asymmetric matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Ensure that the relation is not asymmetric, by making symmetric pairs.
  for _ in range(np.random.randint(1, 5)):
    # Choose 2 pairs randomly.
    i, j = np.random.randint(0, SET_SIZE, 2)
    # Make them symmetric.
    matrix[i][j] = 1
    matrix[j][i] = 1

  return matrix.flatten()

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_binary_dataset(generate_asymmetric_matrix, generate_non_asymmetric_matrix)

In [ ]:
model = get_property_model()

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["accuracy"], history.history["val_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/asymmetry')

# Antisymmetry

In [ ]:
def generate_antisymmetric_matrix() -> np.ndarray:
  """
  Generates an antisymmetric matrix.

  Returns:
    np.ndarray: The generated antisymmetric matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Ensure that the relation is antisymmetric.
  for i in range(SET_SIZE):
    for j in range(SET_SIZE):
      # Skip the main diagonal.
      if i == j:
        continue

      # Ensure antisymmetry.
      if matrix[i][j]:
        matrix[j][i] = 0

  return matrix.flatten()

def generate_non_antisymmetric_matrix() -> np.ndarray:
  """
  Generates a non-antisymmetric matrix.

  Returns:
    np.ndarray: The generated non-antisymmetric matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Ensure that the relation is not antisymmetric, by making some symmetric pairs.
  for _ in range(np.random.randint(1, 5)):
    # Initialize some indices.
    i, j = 0, 0

    # Skip the main diagonal.
    while i == j:
      i, j = np.random.randint(0, SET_SIZE, 2)

    # Make the pairs symmetric.
    matrix[i][j] = 1
    matrix[j][i] = 1

  return matrix.flatten()

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_binary_dataset(generate_antisymmetric_matrix, generate_non_antisymmetric_matrix)

In [ ]:
model = get_property_model()

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["accuracy"], history.history["val_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/antisymmetry')

# Transitivity

In [ ]:
def generate_transitive_matrix() -> np.ndarray:
  """
  Generates a transitive matrix.

  Returns:
    np.ndarray: The generated transitive matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Ensure transitivity.
  for k in range(SET_SIZE):
    for i in range(SET_SIZE):
      for j in range(SET_SIZE):
        if matrix[i][k] and matrix[k][j]:
          matrix[i][j] = 1

  return matrix.flatten()

def generate_non_transitive_matrix() -> np.ndarray:
  """
  Generates a non-transitive matrix.

  Returns:
    np.ndarray: The generated non-transitive matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Violate transitivity.
  for _ in range(np.random.randint(1, 5)):
    # Choose 3 random indices.
    i, j, k = np.random.randint(0, SET_SIZE, 3)

    # Make them violate transitivity.
    matrix[i][j] = 1
    matrix[j][k] = 1
    matrix[i][k] = 0

  return matrix.flatten()

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_binary_dataset(generate_transitive_matrix, generate_non_transitive_matrix)

In [ ]:
model = get_property_model()

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["accuracy"], history.history["val_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/transitivity')

# Antitransitivity

In [ ]:
def generate_antitransitive_matrix() -> np.ndarray:
  """
  Generates an antitransitive matrix.

  Returns:
    np.ndarray: The generated antitransitive matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Ensure antitransitivity.
  for k in range(SET_SIZE):
    for i in range(SET_SIZE):
      for j in range(SET_SIZE):
        if matrix[i][k] and matrix[k][j]:
          matrix[i][j] = 0

  return matrix.flatten()

def generate_non_antitransitive_matrix() -> np.ndarray:
  """
  Generates a non-antitransitive matrix.

  Returns:
    np.ndarray: The generated non-antitransitive matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Violate antitransitivity.
  for _ in range(np.random.randint(1, 5)):
    # Choose 3 random indices.
    i, j, k = np.random.randint(0, SET_SIZE, 3)

    # Make them violate antitransitivity.
    matrix[i][j] = 1
    matrix[j][k] = 1
    matrix[i][k] = 1

  return matrix.flatten()

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_binary_dataset(generate_antitransitive_matrix, generate_non_antitransitive_matrix)

In [ ]:
model = get_property_model()

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["accuracy"], history.history["val_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/antitransitivity')

# Totality

In [ ]:
def generate_totality_matrix() -> np.ndarray:
  """
  Generates a totality matrix.

  Returns:
    np.ndarray: The generated totality matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Ensure totality.
  for i in range(SET_SIZE):
    for j in range(SET_SIZE):
      # This pair already satisfies totality.
      if matrix[i][j] or matrix[j][i]:
        continue

      # Choose randomly which pair gets to satisfy the relation.
      if np.random.randint(0, 2) == 0:
        matrix[i][j] = 1
      else:
        matrix[j][i] = 1

  return matrix.flatten()

def generate_non_totality_matrix() -> np.ndarray:
  """
  Generates a non-totality matrix.

  Returns:
    np.ndarray: The generated non-totality matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Violate totality.
  for _ in range(np.random.randint(1, 5)):
    # Choose 2 indices by random.
    i, j = np.random.randint(0, SET_SIZE, 2)

    # Make them violate totality.
    matrix[i][j] = 0
    matrix[j][i] = 0

  return matrix.flatten()

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_binary_dataset(generate_totality_matrix, generate_non_totality_matrix)

In [ ]:
model = get_property_model()

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["accuracy"], history.history["val_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/totality')

# Trichotomy

In [ ]:
def generate_trichotomy_matrix() -> np.ndarray:
  """
  Generates a trichotomy matrix.

  Returns:
    np.ndarray: The generated trichotomy matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Ensure trichotomy.
  for i in range(SET_SIZE):
    for j in range(SET_SIZE):
      # Skip the main diagonal or when the pair already satisfies trichotomy.
      if i == j or matrix[i][j] or matrix[j][i]:
        continue

      # Choose one of the 2 pairs by random to satisfy the relation.
      if np.random.randint(0, 2) == 0:
        matrix[i][j] = 1
      else:
        matrix[j][i] = 1

  return matrix.flatten()

def generate_non_trichotomy_matrix() -> np.ndarray:
  """
  Generates a non-trichotomy matrix.

  Returns:
    np.ndarray: The generated non-trichotomy matrix.
  """

  # Create a random matrix.
  matrix = get_random_relation()

  # Violate trichotomy.
  for _ in range(np.random.randint(1, 5)):
    # Initialize 2 indices.
    i, j = 0, 0

    # Skip the main diagonal.
    while i == j:
      i, j = np.random.randint(0, SET_SIZE, 2)

    # Make the pairs violate trichotomy.
    matrix[i][j] = 0
    matrix[j][i] = 0

  return matrix.flatten()

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_binary_dataset(generate_trichotomy_matrix, generate_non_trichotomy_matrix)

In [ ]:
model = get_property_model()

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["accuracy"], history.history["val_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/trichotomy')

# Inversion

In [ ]:
def generate_inversion_dataset(number_of_samples: int) -> LabeledData:
  """
  Generates an inversion dataset.

  Args:
    number_of_samples (int): The number of samples to generate.

  Returns:
    LabeledData: The generated inversion dataset.
  """

  # Generate an array of random matrices.
  x = np.random.randint(0, 2, (number_of_samples, SET_SIZE, SET_SIZE))
  # Transpose the array to get the inverse of each matrix.
  y = x.transpose(0, 2, 1)

  # Flatten the matrices.
  x = x.reshape((number_of_samples, SET_SIZE * SET_SIZE))
  y = y.reshape((number_of_samples, SET_SIZE * SET_SIZE))

  return x, y

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_dataset(generate_inversion_dataset)

In [ ]:
model = Sequential()

model.add(Input(shape=(SET_SIZE * SET_SIZE,)))
model.add(Dense(32, activation="relu"))
model.add(Dense(SET_SIZE * SET_SIZE, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[operation_accuracy])

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["operation_accuracy"], history.history["val_operation_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/inversion')

# Composition

In [ ]:
def compose_relation(relation: np.ndarray) -> np.ndarray:
  """
  Composes a relation with itself.

  Args:
    relation (np.ndarray): The relation to compose.

  Returns:
    np.ndarray: The composed relation.
  """

  # Initialize the composed relation.
  composed = np.zeros((SET_SIZE, SET_SIZE))

  # Build the composed relation from the original.
  for i in range(SET_SIZE):
    for j in range(SET_SIZE):
      for k in range(SET_SIZE):
        composed[i][k] = composed[i][k] or (relation[i][j] and relation[j][k])

  return composed

def generate_composition_dataset(number_of_samples: int) -> LabeledData:
  """
  Generates a composition dataset.

  Args:
    number_of_samples (int): The number of samples to generate.

  Returns:
    LabeledData: The generated composition dataset.
  """

  # Create an array of random relations.
  x = np.random.randint(0, 2, (number_of_samples, SET_SIZE, SET_SIZE))

  # Compose each relation with itself and store the new relations in a new array.
  y = np.array([compose_relation(relation) for relation in x])

  # Reshape the x array to fit the model.
  x = x.reshape(number_of_samples, SET_SIZE * SET_SIZE)

  # Reshape the y array to fit the model.
  y = y.reshape(number_of_samples, SET_SIZE * SET_SIZE)

  return x, y

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = generate_full_dataset(generate_composition_dataset)

In [ ]:
model = Sequential()

model.add(Input(shape=(SET_SIZE * SET_SIZE,)))
model.add(Dense(128, activation="relu"))
model.add(Dense(SET_SIZE * SET_SIZE, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[operation_accuracy])

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

In [ ]:
plot_loss(history.history["loss"], history.history["val_loss"])

In [ ]:
plot_accuracy(history.history["operation_accuracy"], history.history["val_operation_accuracy"])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
save_keras_model(model, '/content/composition')